In [48]:
from six.moves.urllib.request import Request, urlopen
from six.moves.urllib.error import HTTPError
from six.moves.urllib.parse import urlencode #probably don't need these
import json
import requests

def authenticate(*args):
    if len(args)>1:
        raise Exception('authenticate takes only 0 or 1 input arguments.') 
    # Save API token if given
    global rinocloud_auth_token
    if len(args) == 1:
        if isinstance(args[0], str):
            rinocloud_auth_token = args[0]
        else:
            raise Exception('Your API token should be entered as a string.') 
    # Return API token if not given an input argument        
    try:
        return 'Token ' + rinocloud_auth_token
    except:
        raise Exception('Set your API token using the authenticate function.') 

API_ROOT = 'http://staging.rinocloud.com/api/1/'
CONNECTION_TIMEOUT = 60


def json_loads_byteified(json_text):
    return _byteify(
        json.loads(json_text, object_hook=_byteify),
        ignore_dicts=True
    )

def _byteify(data, ignore_dicts = False):
    # if this is a unicode string, return its string representation
    if isinstance(data, unicode):
        return data.encode('utf-8')
    # if this is a list of values, return list of byteified values
    if isinstance(data, list):
        return [ _byteify(item, ignore_dicts=True) for item in data ]
    # if this is a dictionary, return dictionary of byteified keys and values
    # but only if we haven't already byteified it
    if isinstance(data, dict) and not ignore_dicts:
        return {
            _byteify(key, ignore_dicts=True): _byteify(value, ignore_dicts=True)
            for key, value in data.iteritems()
        }
    # if it's anything else, return it in its original form
    return data

def dictionary_clean_up(dictionary):
    new_dict = dictionary.copy()
    for key in ['created_on', 'created_on_str', 'file', 'metadata', 'owner', 'project', 'project_name', 'share_code', 'shared', 'size', 'size_str', 'type', 'updated_on', 'updated_on_str' ]:
        new_dict.pop(key, None)
    return new_dict

class RinoRequests(object):

    @classmethod
    def GET(cls, uri, **kw):
        return cls.execute(uri, 'GET', **kw)

    @classmethod
    def POST(cls, uri, **kw):
        return cls.execute(uri, 'POST', **kw)

    @classmethod
    def execute(cls, uri, http_verb, extra_headers=None, _file=None, _json=None, _data=None, _stream = False, **kw):
        url = API_ROOT + uri
        headers = {
        'Authorization': authenticate(),
        'X-Requested-With': 'XMLHttpRequest'
        }
        if http_verb == 'GET':
            r = requests.get(url, json=_json, headers=headers, stream = _stream)
        elif http_verb == 'POST':
            r = requests.post(url, json=_json, files=_file, data=_data, headers=headers)
        return r


class Object(RinoRequests):
    def __init__(self, metadata = {}, file=None, parent = None, tags = None, id=None, **kwargs):
        self.file = file
        self.parent = parent
        self.tags = tags
        self.id = id
        self.metadata = metadata
        self.__dict__.update(metadata)
        self.metadata.update({'parent' : self.parent, 'tags' : self.tags, 'id' : self.id})
        self.__dict__.update(kwargs.pop('Obj_from_dict', ''))
        
            
        
    
        
    def metadata_dict_update(self):
        self.metadata.update({'parent' : self.parent, 'tags' : self.tags, 'id' : self.id})
   
    def add(self, params):
        self.metadata.update(params)
        self.__dict__.update(params)
    
    def upload(self):
        self.metadata_dict_update()
        uri = 'files/upload_multipart/'
        response = self.__class__.POST(uri, _data = {'json': json.dumps(self.metadata)}, _file = {'file': self.file})
        self.__dict__.update(json_loads_byteified(response._content))
        # instead of returning something - update dict.
        # include hidden object to check if upload has already been called and throw error if there is no filepath specified

    def get(self):
        uri = 'files/get_metadata/'
        response = self.__class__.POST(uri, _data = {'id': self.id})
        dictionary = json_loads_byteified(response._content)
        self.metadata = dictionary.get('metadata')
        self.__dict__.update(dictionary)
        self.__dict__.update(self.metadata)

    def update(self):
        self.metadata_dict_update()
        uri = 'files/update_metadata/'
        response = self.__class__.POST(uri, _data = self.metadata_dict)
        dictionary = json_loads_byteified(response._content)
        print dictionary
        print dictionary.get('metadata')
        self.metadata = dictionary.get('metadata')
        self.__dict__.update(dictionary)
        self.__dict__.update(self.metadata)
        
    def download(self, *filename):
        uri = 'files/download/?id=' + str(self.id)
        if filename:
            local_filename = filename
        elif self.name:
            local_filename = self.name
        else:
            self.get()
            local_filename = self.name
        response = self.__class__.GET(uri, _stream=True)
        with open(local_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        return local_filename
    


class Queryset(RinoRequests):

    def __init__(self, dictionary = {}, results = {'results' : 'The query method has not yet been called.'}):
        self.dictionary = dictionary
        self.results = results

    OPERATORS = [
        'lt', 'lte', 'gt', 'gte', 'ne', 'in', 'nin', 'exists', 'or' 
    ]

    @classmethod
    def extract_filter_operator(cls, parameter):
        for op in cls.OPERATORS:
            underscored = '__%s' % op
            if parameter.endswith(underscored):
                return parameter[:-len(underscored)], op
        return parameter, None

    def filter(self, **kw):
        #q = copy.deepcopy(self)
        for name, value in kw.items():
            attr, operator = Queryset.extract_filter_operator(name)
            if operator is None:
                self.dictionary[attr] = value
            elif operator is 'or':
                option_list = []
                for option in value:
                    option_list.append({attr : option})
                self.dictionary['$' + operator] = option_list
            else:
                if attr in self.dictionary:
                    self.dictionary[attr]['$'+ operator] = value
                else:
                    self.dictionary[attr] = {'$'+ operator : value}
                    
        return self

    def query(self):
         uri = 'files/query/'
         response = self.__class__.POST(uri, _json = {'query' : self.dictionary})        
         reply = json_loads_byteified(response._content)
         self.results = []
         for obj in reply['result']:
            self.results.append(Object(Obj_from_dict=obj)) 
         return self.results

        # to add: 
        # update: updates metadata
        # download: downloads file 




In [174]:
obj = Object()

In [175]:
obj.file = open('testcsv.txt', 'r')



In [49]:
authenticate('4db774dcfa4f0985591e3571d89f1062e083e638')

'Token 4db774dcfa4f0985591e3571d89f1062e083e638'

In [167]:
vars(obj)

{'file': {'file': <open file 'testcsv.txt', mode 'r' at 0x104de1f60>},
 'id': 266,
 'metadata': {'id': 266, 'parent': None, 'tags': None},
 'parent': None,
 'tags': None}

In [176]:
obj.upload()

In [172]:
obj.get()

{"id":266,"metadata":{"hh":6},"type":"file","project_name":"eoinisthebest","share_code":null,"owner":"Jamie","size_str":"12.4 kB","updated_on_str":"a day ago","created_on_str":"a day ago","created_on":"2016-03-19T19:32:24.884993Z","updated_on":"2016-03-19T19:56:55.892570Z","shared":false,"name":"testcsv(14).txt","tags":null,"notes":null,"size":12361,"format_string":null,"project":5,"parent":null}


In [177]:
vars(obj)

{'created_on': '2016-03-20T23:26:40.695617Z',
 'created_on_str': 'just now',
 'file': <open file 'testcsv.txt', mode 'r' at 0x104c9e5d0>,
 'format_string': None,
 'id': 268,
 'metadata': {},
 'name': 'testcsv(16).txt',
 'notes': None,
 'owner': 'Jamie',
 'parent': None,
 'project': 5,
 'project_name': 'eoinisthebest',
 'share_code': None,
 'shared': False,
 'size': 12361,
 'size_str': '12.4 kB',
 'tags': None,
 'type': 'file',
 'updated_on': '2016-03-20T23:26:40.695641Z',
 'updated_on_str': 'just now'}

In [159]:
obj.update()

{'updated_on_str': 'just now', 'project_name': 'eoinisthebest', 'name': 'testcsv(14).txt', 'parent': None, 'size_str': '12.4 kB', 'share_code': None, 'notes': None, 'tags': None, 'id': 266, 'project': 5, 'created_on': '2016-03-19T19:32:24.884993Z', 'updated_on': '2016-03-19T19:56:55.892570Z', 'owner': 'Jamie', 'shared': False, 'metadata': {'hh': 6}, 'type': 'file', 'format_string': None, 'created_on_str': '24 minutes ago', 'size': 12361}
{'hh': 6}


In [160]:
vars(obj)

{'created_on': '2016-03-19T19:32:24.884993Z',
 'created_on_str': '24 minutes ago',
 'file': None,
 'format_string': None,
 'hh': 6,
 'id': 266,
 'metadata': {'hh': 6},
 'metadata_dict': {'hh': 6},
 'name': 'testcsv(14).txt',
 'notes': None,
 'owner': 'Jamie',
 'parent': None,
 'project': 5,
 'project_name': 'eoinisthebest',
 'share_code': None,
 'shared': False,
 'size': 12361,
 'size_str': '12.4 kB',
 'tags': None,
 'type': 'file',
 'updated_on': '2016-03-19T19:56:55.892570Z',
 'updated_on_str': 'just now'}

In [50]:
qobj = Queryset()

In [51]:
qobj.filter(key1__gte=1)

In [52]:
res=qobj.query()

In [182]:
type(qobj.results)

dict

In [19]:
obj=Object(Obj_from_dict={'test' : 24, 'test2' : 44})

In [20]:
vars(obj)

{'file': None,
 'id': None,
 'metadata': {'id': None, 'parent': None, 'tags': None},
 'parent': None,
 'tags': None,
 'test': 24,
 'test2': 44}

In [55]:
a=res[1]

In [57]:
a.download()

'testcsv(4).txt'